In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim import corpora
from gensim.models import LdaModel

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word.lower() not in stop_words]
    tokens = [word for word in tokens if word.lower() not in ['quarter', 'year']]
    return tokens

In [ ]:
transcripts = pd.read_csv('datamodel/transcripts.csv')
transcripts.head(1)

In [ ]:
d = {}
for i, row in transcripts.iterrows():
    text_dict = {}
    tid = row['tid']
    tokens = preprocess_text(row['transcript'])
    text_dict[tid] = tokens.copy()
    texts = list(text_dict.values())  # List of preprocessed documents
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    for i, topic in enumerate(lda_model.top_topics(corpus), 1):
        if tid in d:
            d[tid].append({i: topic[0]})
        else:
            d[tid] = [{i: topic[0]}]

In [ ]:
len(d)

In [ ]:
# for topic_num, topic_words in lda_model.print_topics():
#     print(f'Topic #{topic_num + 1}: {topic_words}')

In [ ]:
# cid list_pos(0-4) tid(1-5)
# list(d[554][0][1][0
l = []
for tid, v in d.items():
    for i in range(5):
        for tup in d[tid][i][i+1]:
            l.append([tid, i+1]+list(tup))

In [ ]:
df = pd.DataFrame(l, columns=['tid', 'topic_num', 'value', 'topic'])
df.head()

In [ ]:
len(df)

In [ ]:
df.to_csv('datamodel/ldatopics.csv', index=False)